In [1]:
# import the libraries 
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
import transformers
import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
train_df = pd.read_csv('/kaggle/input/feedback-prize-effectiveness/train.csv')
test_df  = pd.read_csv('/kaggle/input/feedback-prize-effectiveness/test.csv')
train_df.head()

In [3]:
target_column = "discourse_effectiveness"
le = preprocessing.LabelEncoder()
le.fit(train_df[target_column])
train_df['target'] = le.transform(train_df[target_column])

In [4]:
train_df['text'] = train_df['discourse_text']

In [5]:
import nltk ,re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string 
from nltk.stem import WordNetLemmatizer
import numpy as np

nltk.download('wordnet')
nltk.download('stopwords')

In [6]:
emojis = "🍕🐵😑😢🐶️😜😎👊😁😍💖💵👎😀😂🔥😄🏻💥😋👏😱🚌ᴵ͞🌟😊😳😧🙀😐😕👍😮😃😘💩💯⛽🚄😖🏼🚲😟😈💪🙏🎯🌹😇💔😡👌🙄😠😉😤⛺🙂😏🍾🎉😞🏾😅😭👻😥😔😓🏽🎆🍻🍽🎶🌺🤔😪🐰🐇🐱🙆😨🙃💕💗💚🙈😴🏿🤗🇺🇸⤵🏆🎃😩👮💙🐾🐕😆🌠🐟💫💰💎🖐🙅⛲🍰🤐👆🙌💛🙁👀🙊🙉🚬🤓😵😒͝🆕👅👥👄🔄🔤👉👤👶👲🔛🎓😣⏺😌🤑🌏😯😲💞🚓🔔📚🏀👐💤🍇🏡❔⁉👠》🇹🇼🌸🌞🎲😛💋💀🎄💜🤢َِ🗑💃📣👿༼つ༽😰🤣🐝🎅🍺🎵🌎͟🤡🤥😬🤧🚀🤴😝💨🏈😺🌍⏏ệ🍔🐮🍁🍆🍑🌮🌯🤦🍀😫🤤🎼🕺🍸🥂🗽🎇🎊🆘🤠👩🖒🚪🇫🇷🇩🇪😷🇨🇦🌐📺🐋💘💓💐🌋🌄🌅👺🐷🚶🤘ͦ💸👂👃🎫🚢🚂🏃👽😙🎾👹⎌🏒⛸🏄🐀🚑🤷🤙🐒🐈ﷻ🦄🚗🐳👇⛷👋🦊🐽🎻🎹⛓🏹🍷🦆♾🎸🤕🤒⛑🎁🏝🦁🙋😶🔫👁💲🗯👑🚿💡😦🏐🇰🇵👾🐄🎈🔨🐎🤞🐸💟🎰🌝🛳🍭👣🏉💭🎥🐴👨🤳🦍🍩😗🏂👳🍗🕉🐲🍒🐑⏰💊🌤🍊🔹🤚🍎𝑷🐂💅💢💒🚴🖕🖤🥘📍👈➕🚫🎨🌑🐻🤖🎎😼🕷👼📉🍟🍦🌈🔭《🐊🐍🐦🐡💳ἱ🙇🥜🔼"

def remove_emojis(text):
    for emoji in emojis:
        text = text.replace(emoji, '')
    return text

In [8]:
wordnet_lemmatizer = WordNetLemmatizer()
stopwords=stopwords.words('english')
stemmer=PorterStemmer()
# clean unwanted text like stopwords, @(Mention), https(url), #(Hashtag), punctuations
def removeUnwantedText(text):
    #remove urls
    if text == np.NaN or type(text) != str:
      text = " "
    text = re.sub(r'http\S+', " ", text)
    text = re.sub(r'@\w+',' ',text)
    text = re.sub(r'#\w+', ' ', text)
    text = re.sub('r<.*?>',' ', text)
    # html tags
    text = text.lower()
    text = text.split()
    text = " ".join([word for word in text if not word in stopwords])
    for punctuation in string.punctuation:
        text = text.replace(punctuation, "")
    return text

In [9]:
def basic_cleaning(text):
    """
    clear url/ not alpha/ fuck-bitch swear
    """
    text = re.sub(r'https?://www\.\S+\.cm', '', text)
    text = re.sub(r'[^a-zA-Z|\s]', '', text)
    text = re.sub(r'\*+', 'swear', text)
    return text

def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(r'',text)

def remove_emoji(text):
    #emoticons
    #symbols & pictographs
    #transport & map symbols
    #flags (iOS)
    emoji_pattern = re.compile("["\
        u"\U0001F600-\U0001F64F|"\
        u"\U0001F300-\U0001F5FF|"\
        u"\U0001F680-\U0001F6FF|"\
        u"\U0001F1E0-\U0001F1FF|"\
        u"\U00002702-\U000027B0|"\
        u"\U000024C2-\U0001F251"\
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_multiplechars(text):
    """
    for example, so we have “way” instead of “waaaayyyyy”
    """
    text = re.sub(r'(.)\1{3,}', r'\1', text)
    return text

def clean(df):
    for col in ['discourse_text']:#,'selected_text']:
        df[col] = df[col].astype(str).apply(lambda x:basic_cleaning(x))
        df[col] = df[col].astype(str).apply(lambda x:remove_emoji(x))
        df[col] = df[col].astype(str).apply(lambda x:remove_html(x))
        df[col] = df[col].astype(str).apply(lambda x:remove_multiplechars(x))
        df[col] = df[col].astype(str).apply(lambda x:removeUnwantedText(x))
        df[col] = df[col].astype(str).apply(lambda x:removeUnwantedText(x))
        df[col] = df[col].apply(lambda x: remove_emojis(x))
    return df.sample(frac=1)

train_df = clean(train_df)
train_df_selection = train_df.sample(frac=1)
X_tr = train_df_selection.discourse_text.values


test_df = clean(test_df)
test_df_selection = test_df.sample(frac=1)
X_te = test_df_selection.discourse_text.values
print(X_te.shape)
print('clean Done')

In [10]:
test_df_selection

In [11]:
vocab_size = 16000  
maxlen = 64

In [12]:
from tensorflow.keras.preprocessing import text
from tensorflow.keras.preprocessing import sequence

tokenizer = text.Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_df["text"])
def prep_text(texts, tokenizer, max_sequence_length):
    # Turns text into into padded sequences.
    text_sequences = tokenizer.texts_to_sequences(texts)
    return sequence.pad_sequences(text_sequences, maxlen=max_sequence_length)

In [13]:
x= prep_text(train_df['text'],tokenizer,maxlen)
x= np.array(x)
y =np.array(train_df['target'])
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.20, random_state=4)
y_val = tf.one_hot(y_val, 3)
y_train= tf.one_hot(y_train, 3)

In [14]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="gelu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [15]:
embed_dim = 64  # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = 64  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(3, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

In [17]:
model.compile("adam", "categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    x_train, y_train, batch_size=64, epochs=5, validation_data=(x_val, y_val)
)

In [13]:
model.compile("adam", "categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    x_train, y_train, batch_size=32, epochs=5, validation_data=(x_val, y_val)
)

In [ ]:
x_test = prep_text(test_df['discourse_type'],tokenizer,maxlen)
x_test= np.array(x_test)
test_df[target_column] = le.inverse_transform(tf.argmax(model.predict(x_test), axis = 1).numpy())
test_df.to_csv("submission.csv")